<div style="color:#fff; background:#0070c0; font-weight:bold; border:2px solid #0070c0; padding:10px; border-radius:6px;">
📧 <span style="color:#fff;">**Purpose:** This notebook demonstrates how to use <span style="color:yellow;">Spark Expectations</span>.<br>
Its main focus is to show how to pass ROW_DQ rule type and view different action type `drop,ignore,warn` during pipeline execution.</span>
</div>

### Spark - Expectations - User - Guide - Documentation

<div style="color:red; font-weight:bold; border:2px solid red; padding:8px;">
⚠️ ALERT: Notebook meant to be ran by spinning up local docker compose (containers/compose.yaml) !
</div>

* Please read through the [Spark Expectation Documentation](https://engineering.nike.com/spark-expectations) before proceeding with this demo

#### widgets 
* `catalog`, `schema` - leave default values 
  * Tables are going to be prefixed with value provided in user widget text field

<div style="color:orange; font-weight:orange; border:1px solid orange; padding:8px;">
⚠️ Container comes with SparkExpectation by default. If SE version is overriden Kernel will need to be restarted!
</div>

* `Override SE version` check box to install different SparkExpectation library version
* `library_source` combo box defines library url(git branch or pypi) from where to pull library 
  * `pypi` ( installs latest published version available in PyPi)
  * `git` ( installs library from specified git branch)
    * Set `git_branch` input field to match git branch (example `main`)  

In [2]:
# GENERATE INPUT WIDGETS 

import ipywidgets as widgets
from IPython.display import display


widget_user = widgets.Text(
    value='testuser',
    placeholder='Type something',
    description='user: ',
    disabled=False,
    style={'description_width': '100px'}    
)

widget_git_org = widgets.Text(
    value='Nike-Inc',
    placeholder='Type something',
    description='git_org ',
    disabled=False,
    style={'description_width': '100px'}    
)

widget_catalog = widgets.Text(
    value='spark_catalog',
    placeholder='Type something',
    description='catalog:',
    disabled=False,
    style={'description_width': '100px'}    
)

widget_schema = widgets.Text(
    value='default',
    placeholder='Type something',
    description='schema:',
    disabled=False,
    style={'description_width': '100px'}
)

widget_library_source = widgets.Combobox(
    placeholder='Choose source',
    options=['pypi', 'git'],
    description='library_source:',
    ensure_option=True,
    value='git',
    disabled=False,
    style={'description_width': '100px'}
)

widget_git_branch_or_commit = widgets.Text(
    value='main',
    placeholder='Type branch name or commit hash',
    description='git_branch_or_commit:',
    disabled=False,
    style={'description_width': '150px'}
)

widget_override_version = widgets.Checkbox(
    value=False,
    description='Override SE version',
    disabled=False,
    style={'description_width': '30px'}
    
)

hbox = widgets.HBox([
    widget_user,
    widget_catalog, 
    widget_schema,
    widget_override_version, 
    widget_library_source, 
    widget_git_org,
    widget_git_branch_or_commit
])

# Display widgets
display(hbox)


In [ ]:
import re
import pandas as pd

user = re.sub(r'[^a-zA-Z]', '', widget_user.value).lower()
catalog = widget_catalog.value
schema = widget_schema.value
override_se_version = widget_override_version.value
library = widget_library_source.value
org = widget_git_org.value
branch_or_commit = widget_git_branch_or_commit.value

CONFIG = {
    "owner": user,
    "catalog": catalog,
    "schema": schema,
    "user": user,
    "product_id": f"se_{user}_product",
    "in_memory_source": f"se_{user}_source",
    "rules_table": f"{catalog}.{schema}.se_{user}_rules",
    "stats_table": f"{catalog}.{schema}.se_{user}_stats",
    "target_table": f"{catalog}.{schema}.se_{user}_target",
    "override_se_version" : override_se_version,
    "library": library,
    "org": org,
    "branch_or_commit": branch_or_commit
}

config_df = pd.DataFrame(list(CONFIG.items()), columns=['Key', 'Value'])
display(config_df)

# Install Spark Expectation

If Running from local container it will come with latest spark-expectation library


In [ ]:
# Override Spark Expectations based on user input
if override_se_version:
    print("-----OVERRIDING SPARK-EXPECTATIONS VERSION")
    if CONFIG["library"] == "pypi":
      print("-----INSTALLING SPARK-EXPECTATIONS from PyPi")
      %pip install spark-expectations
    elif CONFIG["library"] == "git":
      print(f"-----INSTALLING SPARK-EXPECTATIONS from Git Org/User {CONFIG['org']}, Branch/Commit {CONFIG['branch_or_commit']}")
      giturl = f"git+https://github.com/{CONFIG['org']}/spark-expectations.git@{CONFIG['branch_or_commit']}"
      %pip install --force-reinstall {giturl}    
else:
    print(f"---- Using SparkExpectation from local codebase")

In [ ]:
# Display Spark Expectations installed version
from importlib.metadata import version
print(f"---- Current SparkExpectation Version: {version('spark-expectations')}")

In [ ]:
# CREATE SPARK SESSION AND DATABASE
from pyspark.sql import SparkSession

# Create or get a Spark session
spark = SparkSession.builder \
    .appName("Spark SQL Example") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.0.0") \
    .getOrCreate()

In [ ]:

databases_df = spark.sql("SHOW DATABASES")
databases_df.show(truncate=False)

tables_df = spark.sql("SHOW TABLES")
tables_df.show(truncate=False)


In [ ]:

db_name = f"{CONFIG['catalog']}.{CONFIG['schema']}"
pattern = f"se_{CONFIG['user']}*"

# Set the current catalog
spark.sql(f"USE {CONFIG['catalog']}")

# Drop tables matching pattern
tables_df = spark.sql(f"SHOW TABLES IN {db_name} LIKE '{pattern}'")
tables_to_drop = [row for row in tables_df.collect() if not row["isTemporary"] ]

if tables_to_drop:
    print(f"Found {len(tables_to_drop)} tables to drop.")
    for row in tables_to_drop:
        table_name = row["tableName"]
        spark.sql(f"DROP TABLE IF EXISTS {db_name}.{table_name}")
        print(f"Dropped table: {db_name}.{table_name}")
else:
    print("----- No tables to drop")

# Drop global and local temp views matching pattern

views_df = spark.sql(f"SHOW VIEWS in {db_name} LIKE '{pattern}'")
views_to_drop = views_df.collect()

if views_to_drop:
    print(f"Found {len(views_to_drop)} views to drop.")
    for row in views_to_drop:
        view_name = row["viewName"]
        spark.sql(f"DROP VIEW IF EXISTS {view_name}")
        print(f"Dropped view: {view_name}")
else:
    print("----- No views to drop")

In [ ]:
# Getting Started with Spark Expectations: Simple Example

## 1. Sample Source Dataset
# initialize simple Pandas DataFrame and convert it to a Spark DataFrame

import pandas as pd
from pyspark.sql import SparkSession


## 2. Define Simple `row_dq` Rules
# Create a rules DataFrame with a few simple data quality rules

rules_data = [
    {
        "product_id": CONFIG["product_id"],
        "table_name": CONFIG["target_table"],
        "rule_type": "row_dq",
        "rule": "age_not_null",
        "column_name": "age",
        "expectation": "age IS NOT NULL",
        "action_if_failed": "warn",
        "tag": "completeness",
        "description": "Age must not be null",
        "enable_for_source_dq_validation": True,
        "enable_for_target_dq_validation": True,
        "is_active": True,
        "enable_error_drop_alert": False,
        "error_drop_threshold": 0,
        "priority": "medium",
    },
    {
        "product_id": CONFIG["product_id"],
        "table_name": CONFIG["target_table"],
        "rule_type": "row_dq",
        "rule": "age_adult",
        "column_name": "age",
        "expectation": "age < 20",
        "action_if_failed": "ignore",
        "tag": "validity",
        "description": "Age must be less than 20",
        "enable_for_source_dq_validation": True,
        "enable_for_target_dq_validation": True,
        "is_active": True,
        "enable_error_drop_alert": False,
        "error_drop_threshold": 0,
        "priority": "medium",
    },
        {
        "product_id": CONFIG["product_id"],
        "table_name": CONFIG["target_table"],
        "rule_type": "row_dq",
        "rule": "email_not_null",
        "column_name": "email",
        "expectation": "email IS NOT NULL",
        "action_if_failed": "drop",
        "tag": "completeness",
        "description": "Email must not be null",
        "enable_for_source_dq_validation": True,
        "enable_for_target_dq_validation": True,
        "is_active": True,
        "enable_error_drop_alert": False,
        "error_drop_threshold": 0,
        "priority": "medium",
    }

    
]
rules_df = spark.createDataFrame(pd.DataFrame(rules_data))
rules_df.write.format("delta").mode("overwrite").saveAsTable(CONFIG['rules_table'])

rules_df.show(truncate=False)

In [ ]:
## 3. Run Spark Expectations

from pyspark.sql import DataFrame

from spark_expectations.core import load_configurations
from spark_expectations.config.user_config import Constants as user_config

from spark_expectations.core.expectations import (
    SparkExpectations,
    WrappedDataFrameWriter,
)


writer = WrappedDataFrameWriter().mode("overwrite").format("delta")


# Initialize Default Config 
load_configurations(spark) 
# Custom config (example enable slack/email notifications)
stats_streaming_config_dict = {user_config.se_enable_streaming: False}
notification_conf = {}

"""
This class implements/supports running the data quality rules on a dataframe returned by a function

Args:
    product_id: Name of the product
    rules_df: DataFrame which contains the rules. User is responsible for reading
        the rules_table in which ever system it is
    stats_table: Name of the table where the stats/audit-info need to be written
    debugger: Mark it as "True" if the debugger mode need to be enabled, by default is False
    stats_streaming_options: Provide options to override the defaults, while writing into the stats streaming table
"""
se = SparkExpectations(
    product_id=CONFIG["product_id"],
    rules_df=rules_df,
    stats_table=CONFIG["stats_table"],
    stats_table_writer=writer,
    target_and_error_table_writer=writer,
    stats_streaming_options=stats_streaming_config_dict,
)

#  Initialize input data
data = [
    {"id": 1, "age": 19,   "email": "alice@example.com"},
    {"id": 2, "age": 17,   "email": "bob@example.com"},
    {"id": 3, "age": None, "email": "charlie@example.com"},
    {"id": 4, "age": 40,   "email": "mike@example.com"},
    {"id": 5, "age": None, "email": "ron@example.com"},
    {"id": 6, "age": 35,   "email": None},
]
input_df = spark.createDataFrame(pd.DataFrame(data))
input_df.show(truncate=False)

"""
This decorator helps to wrap a function which returns dataframe and apply dataframe rules on it

Args:
    target_table: Name of the table where the final dataframe need to be written
    write_to_table: Mark it as "True" if the dataframe need to be written as table
    write_to_temp_table: Mark it as "True" if the input dataframe need to be written to the temp table to break
                        the spark plan
    user_conf: Provide options to override the defaults, while writing into the stats streaming table
    target_table_view: This view is created after the _row_dq process to run the target agg_dq and query_dq.
        If value is not provided, defaulted to {target_table}_view
    target_and_error_table_writer: Provide the writer to write the target and error table,
        this will take precedence over the class level writer

Returns:
    Any: Returns a function which applied the expectations on dataset
"""


@se.with_expectations(
    target_table=CONFIG["target_table"],
    write_to_table=True,
    write_to_temp_table=True,
    user_conf=notification_conf,
)
def get_dataset():
    _df_source: DataFrame = input_df
    _df_source.createOrReplaceTempView(CONFIG["in_memory_source"])
    return _df_source


# This will run the DQ checks and raise if any "fail" rules are violated
get_dataset()

%md
# Results

Checkout `Stats` and `Error` table to see spark-expectations execution results

In [ ]:
query_stats_table = f"SELECT * FROM {CONFIG['stats_table']}"
spark.sql(query_stats_table).show(truncate=False)

query_error_table = f"SELECT * FROM {CONFIG['target_table']}_error"
spark.sql(query_error_table).show(truncate=False)

%md
# Display Target Table
Based on the provided rules if `action_if_failed` is set to `drop` target table will remove any rows not passing provided check

In [ ]:
query_target_table = f"""
SELECT *
FROM {CONFIG['target_table']} 
ORDER BY meta_dq_run_id, id
"""

final_data_set_df = spark.sql(query_target_table)

if final_data_set_df is not None:
    final_data_set_df.show(truncate=False)

%md
## Display Removed rows

In [ ]:
input_count = spark.sql(f"SELECT COUNT(*) AS count FROM {CONFIG['in_memory_source']}").collect()[0]['count']
output_count = spark.sql(f"SELECT COUNT(*) AS count FROM {CONFIG['target_table']}").collect()[0]['count']

# Find missing rows in target_table that are present in in_memory_source
removed_rows_df = spark.sql(f"""
SELECT s.*
FROM {CONFIG['in_memory_source']} s
LEFT ANTI JOIN {CONFIG['target_table']} t
ON s.id = t.id
""")

removed_rows_count = removed_rows_df.count()

comparison_df = spark.createDataFrame(
    [
        ("input", input_count),
        ("output", output_count),
        ("removed_records", removed_rows_count)
    ],
    ["table", "record_count"]
)

comparison_df.show()

if removed_rows_count > 0:
    removed_rows_df.show(truncate=False)